## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from Model_Parent import *
from Model_Parent_2 import *

## Importing Data

In [ ]:
df = pd.read_csv("../cleaned_data/auto_mpg_fixed_cleaned.csv", index_col=0)
df.drop('origin', axis=1, inplace=True)

df

## Checking Data

In [ ]:
# Check to make sure that no columns have perfect colinearity
corr_mat = df.corr(method='pearson')
sns.heatmap(corr_mat, cmap='vlag')

## Scaling Data

In [ ]:
# Storing old columns names
df_col_names = list(df.columns)

# Scaling data by min and max in the range of 0 to 1
scaler = MinMaxScaler(feature_range = [0, 1])
tmp = scaler.fit_transform(df)

# Converting scaled values back into dataframe
df = pd.DataFrame(tmp, columns=df_col_names)

df

## Breaking Data in X and y

In [ ]:
X = df.iloc[:,:-1]

In [ ]:
y = df.iloc[:,-1]

# Generating Models

In [ ]:
# Imports to build layered neural nets
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Defining number of folds in cross validation (kfolds method)
cv_folds = 5

# Defining number of epochs
epo = 10

# Defining batch size
bs = 1000

## 2 Layer Neural Networks

### Sigmoid Activation Function

In [ ]:
# nur_list is a list of number of neurons for each layer.
# In this case the first 2 values in the list are neurons of first 2 layers.
# The last value corresponds to the output layer
nur_list = [10, 5, 1]

# a_func is the activation function of choice
a_func = 'sigmoid'

# opt is the optimazation function of choice
opt = 'adam'

# loss_ is the loss funciton of choice
loss_ = 'mean_squared_error'

In [ ]:
nnCrossValidation(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

In [ ]:
nnForwardSelection(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

### Tanh Activation Function

In [ ]:
# nur_list is a list of number of neurons for each layer.
# In this case the first 2 values in the list are neurons of first 2 layers.
# The last value corresponds to the output layer
nur_list = [10, 5, 1]

# a_func is the activation function of choice
a_func = 'tanh'

# opt is the optimazation function of choice
opt = 'adam'

# loss_ is the loss funciton of choice
loss_ = 'mean_squared_error'

In [ ]:
nnCrossValidation(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

In [ ]:
nnForwardSelection(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

### ReLU Activation Function

In [ ]:
# nur_list is a list of number of neurons for each layer.
# In this case the first 2 values in the list are neurons of first 2 layers.
# The last value corresponds to the output layer
nur_list = [10, 5, 1]

# a_func is the activation function of choice
a_func = 'relu'

# opt is the optimazation function of choice
opt = 'adam'

# loss_ is the loss funciton of choice
loss_ = 'mean_squared_error'

In [ ]:
nnCrossValidation(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

In [ ]:
nnForwardSelection(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

## 3 Layer Neural Networks

### Sigmoid Activation Function

In [ ]:
# nur_list is a list of number of neurons for each layer.
# In this case the first 3 values in the list are neurons of first 3 layers.
# The last value corresponds to the output layer
nur_list = [25, 10, 5, 1]

# a_func is the activation function of choice
a_func = 'sigmoid'

# opt is the optimazation function of choice
opt = 'adam'

# loss_ is the loss funciton of choice
loss_ = 'mean_squared_error'

In [ ]:
nnCrossValidation(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

In [ ]:
nnForwardSelection(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

### Tanh Activation Function

In [ ]:
# nur_list is a list of number of neurons for each layer.
# In this case the first 3 values in the list are neurons of first 3 layers.
# The last value corresponds to the output layer
nur_list = [25, 10, 5, 1]

# a_func is the activation function of choice
a_func = 'tanh'

# opt is the optimazation function of choice
opt = 'adam'

# loss_ is the loss funciton of choice
loss_ = 'mean_squared_error'

In [ ]:
nnCrossValidation(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

In [ ]:
nnForwardSelection(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

### ReLU Activation Function

In [ ]:
# nur_list is a list of number of neurons for each layer.
# In this case the first 3 values in the list are neurons of first 3 layers.
# The last value corresponds to the output layer
nur_list = [25, 10, 5, 1]

# a_func is the activation function of choice
a_func = 'relu'

# opt is the optimazation function of choice
opt = 'adam'

# loss_ is the loss funciton of choice
loss_ = 'mean_squared_error'

In [ ]:
nnCrossValidation(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

In [ ]:
nnForwardSelection(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

## 5 Layer Neural Networks

### Sigmoid Activation Function

In [ ]:
# nur_list is a list of number of neurons for each layer.
# In this case the first 5 values in the list are neurons of first 5 layers.
# The last value corresponds to the output layer
nur_list = [500, 100, 25, 10, 5, 1]

# a_func is the activation function of choice
a_func = 'sigmoid'

# opt is the optimazation function of choice
opt = 'adam'

# loss_ is the loss funciton of choice
loss_ = 'mean_squared_error'

In [ ]:
nnCrossValidation(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

In [ ]:
nnForwardSelection(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

### Tanh Activation Function

In [ ]:
# nur_list is a list of number of neurons for each layer.
# In this case the first 5 values in the list are neurons of first 5 layers.
# The last value corresponds to the output layer
nur_list = [500, 100, 25, 10, 5, 1]

# a_func is the activation function of choice
a_func = 'tanh'

# opt is the optimazation function of choice
opt = 'adam'

# loss_ is the loss funciton of choice
loss_ = 'mean_squared_error'

In [ ]:
nnCrossValidation(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

In [ ]:
nnForwardSelection(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

### ReLU Activation Function

In [ ]:
# nur_list is a list of number of neurons for each layer.
# In this case the first 5 values in the list are neurons of first 5 layers.
# The last value corresponds to the output layer
nur_list = [500, 100, 25, 10, 5, 1]

# a_func is the activation function of choice
a_func = 'relu'

# opt is the optimazation function of choice
opt = 'adam'

# loss_ is the loss funciton of choice
loss_ = 'mean_squared_error'

In [ ]:
nnCrossValidation(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)

In [ ]:
nnForwardSelection(X, y, cv_folds, epo, bs, nur_list, a_func, opt, loss_)